In [1]:
import pandas as pd
df = pd.read_csv("Sales.csv")

In [2]:
import pandas as pd

unique_pizza_ids = df['pizza_name_id'].unique()

dataframes = {}

for pizza_id in unique_pizza_ids:
    df_filtered = df[df['pizza_name_id'] == pizza_id][['week_number', 'quantity', 'order_date']]
    dataframes[pizza_id] = df_filtered

In [3]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error

results = {}



example_pizza_id = unique_pizza_ids[0]
df = dataframes[example_pizza_id]




c:\Users\ANNIE CHRISTINA G\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [4]:
df

,week_number,quantity,order_date
0,1,1,2015-01-01
178,1,1,2015-01-02
356,1,1,2015-01-03
388,1,1,2015-01-03
567,1,1,2015-01-04
...,...,...,...
48056,52,1,2015-12-26
48112,52,1,2015-12-27
48211,53,1,2015-12-28
48238,53,1,2015-12-28


In [5]:
df_weekly = df.groupby('week_number').sum().reset_index()

In [6]:
df_weekly

,week_number,quantity,order_date
0,1,5,2015-01-012015-01-022015-01-032015-01-032015-0...
1,2,5,2015-01-062015-01-062015-01-072015-01-092015-0...
2,3,3,2015-01-142015-01-172015-01-17
3,4,4,2015-01-192015-01-212015-01-222015-01-23
4,5,11,2015-01-272015-01-272015-01-282015-01-292015-0...
5,6,11,2015-02-032015-02-032015-02-042015-02-042015-0...
6,7,8,2015-02-102015-02-112015-02-122015-02-122015-0...
7,8,12,2015-02-162015-02-162015-02-162015-02-162015-0...
8,9,8,2015-02-232015-02-242015-02-242015-02-242015-0...
9,10,18,2015-03-022015-03-022015-03-022015-03-022015-0...


In [7]:
df_weekly.rename(columns={'week_number': 'ds', 'quantity': 'y'}, inplace=True)

In [8]:
df_weekly

,ds,y,order_date
0,1,5,2015-01-012015-01-022015-01-032015-01-032015-0...
1,2,5,2015-01-062015-01-062015-01-072015-01-092015-0...
2,3,3,2015-01-142015-01-172015-01-17
3,4,4,2015-01-192015-01-212015-01-222015-01-23
4,5,11,2015-01-272015-01-272015-01-282015-01-292015-0...
5,6,11,2015-02-032015-02-032015-02-042015-02-042015-0...
6,7,8,2015-02-102015-02-112015-02-122015-02-122015-0...
7,8,12,2015-02-162015-02-162015-02-162015-02-162015-0...
8,9,8,2015-02-232015-02-242015-02-242015-02-242015-0...
9,10,18,2015-03-022015-03-022015-03-022015-03-022015-0...


In [9]:
df_weekly['ds'] = pd.date_range(start=df['order_date'].min(), periods=len(df_weekly), freq='W')

In [10]:
df_weekly

,ds,y,order_date
0,2015-01-04,5,2015-01-012015-01-022015-01-032015-01-032015-0...
1,2015-01-11,5,2015-01-062015-01-062015-01-072015-01-092015-0...
2,2015-01-18,3,2015-01-142015-01-172015-01-17
3,2015-01-25,4,2015-01-192015-01-212015-01-222015-01-23
4,2015-02-01,11,2015-01-272015-01-272015-01-282015-01-292015-0...
5,2015-02-08,11,2015-02-032015-02-032015-02-042015-02-042015-0...
6,2015-02-15,8,2015-02-102015-02-112015-02-122015-02-122015-0...
7,2015-02-22,12,2015-02-162015-02-162015-02-162015-02-162015-0...
8,2015-03-01,8,2015-02-232015-02-242015-02-242015-02-242015-0...
9,2015-03-08,18,2015-03-022015-03-022015-03-022015-03-022015-0...


In [11]:

split_point = int(len(df_weekly) * 0.7)
train_df = df_weekly[:split_point]

test_df = df_weekly[split_point:]

In [14]:
test_df

,ds,y,order_date
37,2015-09-20,8,2015-09-142015-09-152015-09-162015-09-162015-0...
38,2015-09-27,6,2015-09-212015-09-212015-09-232015-09-232015-0...
39,2015-10-04,8,2015-09-292015-09-302015-10-012015-10-012015-1...
40,2015-10-11,9,2015-10-062015-10-062015-10-082015-10-082015-1...
41,2015-10-18,8,2015-10-152015-10-152015-10-152015-10-162015-1...
42,2015-10-25,8,2015-10-202015-10-212015-10-212015-10-222015-1...
43,2015-11-01,3,2015-10-302015-10-312015-11-01
44,2015-11-08,3,2015-11-032015-11-042015-11-07
45,2015-11-15,13,2015-11-092015-11-092015-11-092015-11-092015-1...
46,2015-11-22,9,2015-11-162015-11-172015-11-182015-11-182015-1...


In [15]:
model = Prophet()

model.fit(train_df)

future = model.make_future_dataframe(periods=len(test_df), freq='W')

10:32:50 - cmdstanpy - INFO - Chain [1] start processing
10:32:50 - cmdstanpy - INFO - Chain [1] done processing


In [16]:
future

,ds
0,2015-01-04
1,2015-01-11
2,2015-01-18
3,2015-01-25
4,2015-02-01
5,2015-02-08
6,2015-02-15
7,2015-02-22
8,2015-03-01
9,2015-03-08


In [18]:

forecast = model.predict(future)
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2015-01-04,8.035560,3.992152,12.000875,8.035560,8.035560,0.0,0.0,0.0,0.0,0.0,0.0,8.035560
1,2015-01-11,8.116812,3.998008,12.444889,8.116812,8.116812,0.0,0.0,0.0,0.0,0.0,0.0,8.116812
2,2015-01-18,8.198082,3.973136,12.235165,8.198082,8.198082,0.0,0.0,0.0,0.0,0.0,0.0,8.198082
3,2015-01-25,8.279352,4.065340,12.485858,8.279352,8.279352,0.0,0.0,0.0,0.0,0.0,0.0,8.279352
4,2015-02-01,8.360622,3.706741,12.325495,8.360622,8.360622,0.0,0.0,0.0,0.0,0.0,0.0,8.360622
5,2015-02-08,8.441892,4.192292,12.617610,8.441892,8.441892,0.0,0.0,0.0,0.0,0.0,0.0,8.441892
6,2015-02-15,8.523162,4.150100,12.741620,8.523162,8.523162,0.0,0.0,0.0,0.0,0.0,0.0,8.523162
7,2015-02-22,8.604432,4.578737,12.687637,8.604432,8.604432,0.0,0.0,0.0,0.0,0.0,0.0,8.604432
8,2015-03-01,8.685702,4.417211,12.789517,8.685702,8.685702,0.0,0.0,0.0,0.0,0.0,0.0,8.685702
9,2015-03-08,8.766972,4.715271,12.876300,8.766972,8.766972,0.0,0.0,0.0,0.0,0.0,0.0,8.766972


In [19]:

test_forecast = forecast.iloc[split_point:split_point + len(test_df)][['ds', 'yhat']]

In [20]:
test_forecast

,ds,yhat
37,2015-09-20,11.042529
38,2015-09-27,11.123799
39,2015-10-04,11.205069
40,2015-10-11,11.286339
41,2015-10-18,11.367609
42,2015-10-25,11.448879
43,2015-11-01,11.530149
44,2015-11-08,11.611419
45,2015-11-15,11.692689
46,2015-11-22,11.773959


In [ ]:



mape = mean_absolute_percentage_error(test_df['y'], test_forecast['yhat'])

week_forecast = forecast.tail(1)[['ds', 'yhat']]

results[f'unique_pizza_ids[{j}]'] = {

    'MAPE': mape,

    'Next_Week_Prediction': week_forecast

}



In [ ]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error

results = {}

for j in range(0,91):
    example_pizza_id = unique_pizza_ids[j]
    df = dataframes[example_pizza_id]
    df_weekly = df.groupby('week_number').sum().reset_index()
    df_weekly.rename(columns={'week_number': 'ds', 'quantity': 'y'}, inplace=True)
    df_weekly['ds'] = pd.date_range(start=df['order_date'].min(), periods=len(df_weekly), freq='W')

    split_point = int(len(df_weekly) * 0.7)
    train_df = df_weekly[:split_point]
    test_df = df_weekly[split_point:]

    model = Prophet()
    model.fit(train_df)

    future = model.make_future_dataframe(periods=len(test_df), freq='W')
    forecast = model.predict(future)

    test_forecast = forecast.iloc[split_point:split_point + len(test_df)][['ds', 'yhat']]
    mape = mean_absolute_percentage_error(test_df['y'], test_forecast['yhat'])

    week_forecast = forecast.tail(1)[['ds', 'yhat']]

    results[f'unique_pizza_ids[{j}]'] = {
        'MAPE': mape,
        'Next_Week_Prediction': week_forecast
    }

    

In [ ]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error

results = {}

example_pizza_id = unique_pizza_ids[j]

df = dataframes[example_pizza_id]

df_weekly = df.groupby('week_number').sum().reset_index()

df_weekly.rename(columns={'week_number': 'ds', 'quantity': 'y'}, inplace=True)

df_weekly['ds'] = pd.date_range(start=df['order_date'].min(), periods=len(df_weekly), freq='W')

split_point = int(len(df_weekly) * 0.7)

train_df = df_weekly[:split_point]

test_df = df_weekly[split_point:]

model = Prophet()

model.fit(train_df)

future = model.make_future_dataframe(periods=len(test_df), freq='W')

forecast = model.predict(future)

test_forecast = forecast.iloc[split_point:split_point + len(test_df)][['ds', 'yhat']]

mape = mean_absolute_percentage_error(test_df['y'], test_forecast['yhat'])

week_forecast = forecast.tail(1)[['ds', 'yhat']]

results[f'unique_pizza_ids[{j}]'] = {

    'MAPE': mape,

    'Next_Week_Prediction': week_forecast

}

 